In [ ]:
import json  
import matplotlib.pyplot as plt

In [ ]:
## get data first
from datasets import load_dataset
submissions = load_dataset("microsoft/llmail-inject-challenge")['Phase1']
jobs = [submissions._getitem(i) for i in range(len(submissions))]

/disk1/miniconda3/envs/satml_comp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### For which retrieval level?  (leave empty if for all)

In [80]:
retrieval_level = ''

## For each team: 1) check the solved levels and their times (first solve)

## Also check for each level which teams solved it 

In [61]:
from datetime import datetime


#### all jobs that solved a level per team ####
teams_to_solves = dict()

#### map levels to teams whol solves it 
levels_to_team_solved = dict()

for job in jobs:
    if job['output']:
        obj = json.loads(job['objectives']) 
        if obj:
            ## successful job 
            if all(obj.values()):
                if not job['team_id'] in teams_to_solves: teams_to_solves[job['team_id']] = dict()
                if not job['scenario'] in teams_to_solves[job['team_id']]: teams_to_solves[job['team_id']][job['scenario']] = []
                teams_to_solves[job['team_id']][job['scenario']].append(datetime.strptime(job['completed_time'],"%Y-%m-%dT%H:%M:%S.%f%z"))
                
                if not job['scenario'] in levels_to_team_solved: levels_to_team_solved[job['scenario']] = set()
                levels_to_team_solved[job['scenario']].add(job['team_id'])
                


#### first solve for each team in each level ####
teams_to_first_solve = dict()
for team in teams_to_solves:
    teams_to_first_solve[team] = dict()
    for level in teams_to_solves[team]:
        first_solve = min(teams_to_solves[team][level])
        teams_to_first_solve[team][level] = first_solve


#### initialize attempts before solve ####
teams_to_attempts_before_solve = dict()
for team in teams_to_solves:
    teams_to_attempts_before_solve[team] = dict()
    for level in teams_to_solves[team]:
        teams_to_attempts_before_solve[team][level] = 0 



## For each job: 
### map it to which team submitted it 
### check if that team has solved it
#### if yes, check if the job time is less than the solve time
##### if yes, increment the trials before solve for that team and that level. 

In [66]:
for job in jobs: 
    obj = json.loads(job['objectives'])
    if not obj: continue
    team_id = job['team_id']
    scenario = job['scenario']
    job_completed_time = datetime.strptime(job['completed_time'],"%Y-%m-%dT%H:%M:%S.%f%z")
    
    #scenario ever solved?
    if scenario in levels_to_team_solved:
        #team solved it?
        if team_id in levels_to_team_solved[scenario]:
            #less than the first solve? 
            if job_completed_time <  teams_to_first_solve[team_id][scenario]:
                    teams_to_attempts_before_solve[team_id][scenario] += 1


## for each defense:
### get a list of trials_before_success for each team who solved it and average it

In [81]:
trials_before_success_per_defense = {'prompt_shield': [], 'task_tracker': [], 'llm_judge': []}

prompt_shield = ['a', 'b']
tasktracker = ['c', 'd']
llm_judge = ['g', 'h']

def check_defense(scenario, defense_letters): 
    for letter in defense_letters: 
        if letter in scenario: return True 
    return False 


for scenario in levels_to_team_solved:
    for team in teams_to_attempts_before_solve:
        if scenario in teams_to_attempts_before_solve[team]:
            trials_before_succes_for_team = teams_to_attempts_before_solve[team][scenario]
            if not retrieval_level in scenario: continue 
            scenario_  = scenario.split('level')[-1]
            if check_defense(scenario_,prompt_shield):
                trials_before_success_per_defense['prompt_shield'].append(trials_before_succes_for_team)
            elif check_defense(scenario_,tasktracker):
                trials_before_success_per_defense['task_tracker'].append(trials_before_succes_for_team)
            elif check_defense(scenario_,llm_judge):
                trials_before_success_per_defense['llm_judge'].append(trials_before_succes_for_team)

import numpy as np 

print(f'prompt shield - mean: {np.mean(trials_before_success_per_defense['prompt_shield'])}, std: {np.std(trials_before_success_per_defense['prompt_shield'])}')


print(f'task tracker - mean: {np.mean(trials_before_success_per_defense['task_tracker'])}, std: {np.std(trials_before_success_per_defense['task_tracker'])}')


print(f'llm judge - mean: {np.mean(trials_before_success_per_defense['llm_judge'])}, std: {np.std(trials_before_success_per_defense['llm_judge'])}')

prompt shield - mean: 88.00611620795107, std: 426.2277965116387
task tracker - mean: 158.31336405529953, std: 518.5892627715648
llm judge - mean: 505.23595505617976, std: 948.4642823464726
